In [1]:
# 축구를 잘하는 사람이 족구도 잘한다.
# 요리를 잘하는 사람이 라면도 잘 끓인다
# 파이썬을 공부한 사람은 다른 언어를 잘 할 가능성이 많다.

In [2]:
# 딥러닝에도 적용
# 자연영상으로 학습한 신경망을 새나 개의 종을 분류하거나 자율주행에서 차선과 보행자 인식하는사용

In [3]:
# 어떤 도메인의 데이터로 학습한 모델을 다른 도메인의 데이터를 인식하는데 활용해서 성능 향상을 도모하는 기법
# cub200-2011 : 200종의 새 영상을 가지는데 부류별로 60장 밖에 안되서 과잉적합우려
# ImageNet으로 예비학습된 모델을 cub200-2011로 전이하는 전이학습

In [46]:
#ImageNet으로 학습된 ResNet50을 cub 데이터셋으로 전이 학습
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
import os

In [48]:
import zipfile
zippath = "/content/drive/MyDrive/CUB200/CUB200.zip"
zip_ref = zipfile.ZipFile(zippath,'r')
zip_ref.extractall('/content/drive/MyDrive/CUB200')  
zip_ref.close()
# !unzip -qq "/content/drive/MyDrive/CUB200/CUB200.zip"

In [49]:
class_reduce = 0.1
train_folder = "/content/drive/MyDrive/CUB200/train"
test_folder = "/content/drive/MyDrive/CUB200/test"

In [50]:
# 정답의 라벨수(부류의 갯수)
no_class = int(len(os.listdir(test_folder))*class_reduce)

In [ ]:
import glob
# 현재경로 뿐만 아니라 특정경로의 디렉터리명이나 확장자를 지정하면 해당 파일만
# 가져오는데 절대경로로 가져온다.
glob.glob(train_folder+"/*")

In [53]:
def convertXYData(foldername):
  x_train,y_train = [],[]
  for i, class_name in  enumerate(os.listdir(foldername)):        
    if i < no_class:      
      for fname in os.listdir(foldername+'/'+class_name):        
        img = image.load_img(foldername+'/'+class_name+'/'+fname,target_size=(224,224) )        
        if len(img.getbands()) != 3:
          print(f"유효하지 않은 영상발생 {class_name} {fname}")
          continue
        x = image.img_to_array(img)
        x = preprocess_input(x)
        x_train.append(x)
        y_train.append(i)
  return (x_train,y_train)

In [ ]:
x_train,y_train = convertXYData(train_folder)

In [54]:
x_test,y_test = convertXYData(test_folder)

In [56]:
np.array(x_train).shape, np.array(x_test).shape

((600, 224, 224, 3), (515, 224, 224, 3))

In [57]:
np.array(y_train).shape, np.array(y_test).shape

((600,), (515,))

In [60]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test =  np.asarray(x_test)
y_test = np.asarray(y_test)

In [64]:
y_train = tf.keras.utils.to_categorical(y_train,no_class)
y_test = tf.keras.utils.to_categorical(y_test,no_class)

In [67]:
# help(ResNet50)
base_model = ResNet50(weights='imagenet',input_shape=(224, 224, 3), include_top = False)

94781440/94765736 [==============================] - 0s 0us/step


In [69]:
######################  메인 알고리즘... #############
cnn = Sequential()
cnn.add(base_model)
### 끝
## 분류를 하기위해서 FC
cnn.add(Flatten())
cnn.add(Dense(1024, activation='relu'))
cnn.add(Dense(no_class,activation='softmax'))